In [31]:
import pandas as pd
import json
import http.client
import time
from json.decoder import JSONDecodeError

In [2]:
pd.options.display.max_columns = None

In [3]:
conn = http.client.HTTPSConnection("free-nba.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "06a7d125fdmsh1f549e389f7fe84p17c7cajsn8f40891f4a76",
    'X-RapidAPI-Host': "free-nba.p.rapidapi.com"
}

conn.request("GET", "/players?per_page=25&page=0", headers=headers)

res = conn.getresponse()
data = res.read()

# JSON to a Python dictionary
json_data = json.loads(data.decode("utf-8"))

# dataframe from dictionary
df = pd.DataFrame(json_data['data'])

print(df)


      id first_name  height_feet  height_inches   last_name position  \
0     14        Ike          NaN            NaN    Anigbogu        C   
1     25        Ron          NaN            NaN       Baker        G   
2     47     Jabari          NaN            NaN        Bird        G   
3     67    MarShon          NaN            NaN      Brooks        G   
4     71    Lorenzo          NaN            NaN       Brown        G   
5     90       Omri          NaN            NaN      Casspi        F   
6      1       Alex          6.0            6.0     Abrines        G   
7    119      Tyler          NaN            NaN       Davis        C   
8    149     Keenan          NaN            NaN       Evans        G   
9    179     Marcin          NaN            NaN      Gortat        C   
10  1593     Andrew          NaN            NaN       Bogut        F   
11   241       Amir          6.0            9.0     Johnson      C-F   
12   392    Malachi          NaN            NaN  Richardson     

In [5]:
df.describe()

,id,height_feet,height_inches,weight_pounds
count,25.000000,3.000000,3.000000,3.000000
mean,319.320000,6.333333,5.000000,228.333333
std,318.854685,0.577350,4.582576,24.664414
min,1.000000,6.000000,0.000000,200.000000
25%,90.000000,6.000000,3.000000,220.000000
50%,281.000000,6.000000,6.000000,240.000000
75%,437.000000,6.500000,7.500000,242.500000
max,1593.000000,7.000000,9.000000,245.000000


In [32]:
df_list = []
conn.close()
for i in range(10000):
    try:
        conn.request("GET", f"/players?per_page=100&page={i}", headers=headers)
        res = conn.getresponse()
        data = res.read()
        data = json.loads(data.decode("utf-8"))
        df = pd.json_normalize(data['data'])
        df_list.append(df)
    except JSONDecodeError:
        continue
    except Exception as e:
        print(f"An error occurred: {e}")
        continue

all_players = pd.concat(df_list, ignore_index=True)

In [33]:
all_players.describe()

,id,team.id
count,5.230000e+03,5230.000000
mean,1.328451e+07,16.822180
std,1.996876e+07,10.878144
min,1.000000e+00,1.000000
25%,1.209250e+03,8.000000
50%,2.515500e+03,16.000000
75%,3.801773e+07,24.000000
max,4.824849e+07,51.000000


In [13]:
all_players.sample(20)

,id,first_name,height_feet,height_inches,last_name,position,weight_pounds,team.id,team.abbreviation,team.city,team.conference,team.division,team.full_name,team.name
3462,666953,Quinndary,NaN,NaN,Weatherspoon,G,NaN,10,GSW,Golden State,West,Pacific,Golden State Warriors,Warriors
4715,3547251,Cole,None,None,Anthony,G,None,22,ORL,Orlando,East,Southeast,Orlando Magic,Magic
1502,1881,Pooh,None,None,Jeter,,None,26,SAC,Sacramento,West,Pacific,Sacramento Kings,Kings
1459,1838,Rodrigue,None,None,Beaubois,,None,7,DAL,Dallas,West,Southwest,Dallas Mavericks,Mavericks
3542,46390753,Stan,NaN,NaN,Brown,,NaN,10,GSW,Golden State,West,Pacific,Golden State Warriors,Warriors
3885,46392457,Chuck,None,None,Cooper,,None,2,BOS,Boston,East,Atlantic,Boston Celtics,Celtics
3667,46390772,Bill,NaN,NaN,Roberts,,NaN,38,BOM,,,,St. Louis Bombers,St. Louis Bombers
3000,24489019,Jarron,NaN,NaN,Cumberland,G,NaN,25,POR,Portland,West,Northwest,Portland Trail Blazers,Trail Blazers
2264,2684,Brant,None,None,Weidner,,None,27,SAS,San Antonio,West,Southwest,San Antonio Spurs,Spurs
3515,19465586,Kevin,NaN,NaN,Pangos,G,NaN,6,CLE,Cleveland,East,Central,Cleveland Cavaliers,Cavaliers


In [14]:
all_players[all_players['first_name'] == 'LeBron']

,id,first_name,height_feet,height_inches,last_name,position,weight_pounds,team.id,team.abbreviation,team.city,team.conference,team.division,team.full_name,team.name
4058,237,LeBron,6.0,8.0,James,F,250.0,14,LAL,Los Angeles,West,Pacific,Los Angeles Lakers,Lakers


Starting loop through entire stats endpoint with 5 second pauses to avoid timeouts.

In [15]:
# conn = http.client.HTTPSConnection("free-nba.p.rapidapi.com")
# headers = {
#     'X-RapidAPI-Key': "06a7d125fdmsh1f549e389f7fe84p17c7cajsn8f40891f4a76",
#     'X-RapidAPI-Host': "free-nba.p.rapidapi.com"
# }
# stats_list = []
# conn.close()
# for i in range(3100):
#     for page in range(i * 100, (i + 1) * 100):
#         conn.request("GET", f"/stats?per_page=100&page={page}", headers=headers)
#         res = conn.getresponse()
#         data = res.read().decode("utf-8")
#         try:
#             data = json.loads(data)
#             stats = pd.json_normalize(data['data'])
#             stats_list.append(stats)
#         except json.JSONDecodeError:
#             print(f"Error decoding page {page}")
#             time.sleep(5)
#     print(f"Finished iteration {i}")
#     time.sleep(5)

Finished iteration 0
Error decoding page 105
Finished iteration 1
Error decoding page 219
Error decoding page 220
Error decoding page 221
Finished iteration 2
Error decoding page 324
Error decoding page 326
Error decoding page 327
Error decoding page 329
Finished iteration 3
Error decoding page 443
Error decoding page 446
Finished iteration 4
Error decoding page 560
Error decoding page 561
Finished iteration 5
Finished iteration 6
Error decoding page 763
Error decoding page 764
Finished iteration 7
Error decoding page 862
Error decoding page 863
Error decoding page 864
Error decoding page 865
Finished iteration 8
Error decoding page 976
Error decoding page 977
Error decoding page 980
Finished iteration 9
Error decoding page 1088
Error decoding page 1089
Error decoding page 1091
Finished iteration 10
Finished iteration 11
Finished iteration 12
Error decoding page 1310
Error decoding page 1313
Error decoding page 1316
Finished iteration 13
Error decoding page 1431
Error decoding page 143

Error decoding page 15828
Error decoding page 15830
Error decoding page 15831
Error decoding page 15832
Finished iteration 158
Error decoding page 15946
Error decoding page 15949
Error decoding page 15950
Error decoding page 15952
Finished iteration 159
Error decoding page 16098
Error decoding page 16099
Finished iteration 160
Finished iteration 161
Error decoding page 16215
Error decoding page 16218
Finished iteration 162
Finished iteration 163
Error decoding page 16411
Error decoding page 16414
Error decoding page 16415
Error decoding page 16416
Finished iteration 164
Error decoding page 16531
Error decoding page 16532
Error decoding page 16533
Error decoding page 16534
Error decoding page 16535
Finished iteration 165
Error decoding page 16646
Error decoding page 16647
Error decoding page 16648
Error decoding page 16651
Finished iteration 166
Error decoding page 16748
Error decoding page 16750
Error decoding page 16751
Error decoding page 16754
Error decoding page 16755
Finished iter

Error decoding page 23426
Error decoding page 23427
Error decoding page 23428
Finished iteration 234
Error decoding page 23545
Error decoding page 23546
Error decoding page 23550
Error decoding page 23551
Finished iteration 235
Error decoding page 23655
Error decoding page 23656
Error decoding page 23658
Error decoding page 23660
Error decoding page 23663
Finished iteration 236
Error decoding page 23771
Error decoding page 23772
Error decoding page 23775
Finished iteration 237
Error decoding page 23893
Error decoding page 23894
Error decoding page 23896
Error decoding page 23898
Finished iteration 238
Finished iteration 239
Error decoding page 24016
Error decoding page 24017
Error decoding page 24018
Finished iteration 240
Finished iteration 241
Error decoding page 24206
Error decoding page 24209
Error decoding page 24210
Error decoding page 24211
Error decoding page 24212
Finished iteration 242
Error decoding page 24310
Error decoding page 24311
Error decoding page 24312
Error decodin

Error decoding page 31019
Finished iteration 310
Error decoding page 31137
Error decoding page 31139
Finished iteration 311
Error decoding page 31241
Error decoding page 31242
Error decoding page 31245
Error decoding page 31247
Finished iteration 312
Error decoding page 31365
Error decoding page 31368
Error decoding page 31370
Finished iteration 313
Error decoding page 31478
Finished iteration 314
Error decoding page 31591
Error decoding page 31593
Finished iteration 315
Error decoding page 31686
Error decoding page 31687
Error decoding page 31688
Error decoding page 31689
Error decoding page 31690
Finished iteration 316
Finished iteration 317
Error decoding page 31811
Error decoding page 31812
Finished iteration 318
Error decoding page 31914
Error decoding page 31916
Error decoding page 31917
Error decoding page 31918
Finished iteration 319
Error decoding page 32036
Error decoding page 32038
Finished iteration 320
Error decoding page 32154
Error decoding page 32155
Finished iteration 

KeyboardInterrupt: 

In [16]:
# stats = pd.concat(stats_list, ignore_index=True)

In [17]:
# stats.describe()

,id,game.id,game.home_team_id,game.home_team_score,game.period,game.season,game.visitor_team_id,game.visitor_team_score,player.id,player.team_id,team.id,player
count,1.530691e+06,1.530691e+06,1.530691e+06,1.530691e+06,1.530691e+06,1.530691e+06,1.530691e+06,1.530691e+06,1.530681e+06,1.530681e+06,1.530691e+06,0.0
mean,3.913549e+06,2.410644e+05,1.554807e+01,1.050423e+02,4.005784e+00,1.994913e+03,1.559306e+01,1.012832e+02,7.822644e+06,1.564297e+01,1.557188e+01,NaN
std,5.502326e+06,3.624515e+05,9.339840e+00,1.474729e+01,1.065776e-01,1.923856e+01,9.399572e+00,1.433899e+01,1.716143e+07,9.542807e+00,9.358309e+00,NaN
min,1.000000e+00,1.000000e+00,1.000000e+00,1.800000e+01,0.000000e+00,1.946000e+03,1.000000e+00,1.900000e+01,1.000000e+00,1.000000e+00,1.000000e+00,NaN
25%,3.911830e+05,1.733000e+04,8.000000e+00,9.500000e+01,4.000000e+00,1.982000e+03,8.000000e+00,9.200000e+01,6.460000e+02,8.000000e+00,8.000000e+00,NaN
50%,7.859130e+05,3.345500e+04,1.500000e+01,1.050000e+02,4.000000e+00,1.998000e+03,1.500000e+01,1.010000e+02,1.561000e+03,1.400000e+01,1.500000e+01,NaN
75%,8.947775e+06,4.737940e+05,2.300000e+01,1.150000e+02,4.000000e+00,2.011000e+03,2.300000e+01,1.110000e+02,2.948000e+03,2.300000e+01,2.300000e+01,NaN
max,1.389861e+07,9.075270e+05,5.100000e+01,1.790000e+02,8.000000e+00,2.022000e+03,5.100000e+01,1.760000e+02,4.824849e+07,5.100000e+01,5.100000e+01,NaN


In [18]:
# page, i

(37982, 379)

In [20]:
# stats.to_pickle('stats_p0_p37982_i379.pkl')

In [5]:
# stats = pd.read_pickle('stats_p0_p37982_i379.pkl')

Testing using a separate dataframe of current 2023 players, and looping through to only pull the stats of games of said players.

In [25]:
act_plyrs_df = pd.read_csv('active_players.csv')

In [28]:
act_plyrs_df.drop('Unnamed: 0', axis=1, inplace=True)

In [19]:
conn = http.client.HTTPSConnection("free-nba.p.rapidapi.com")
headers = {
    'X-RapidAPI-Key': "06a7d125fdmsh1f549e389f7fe84p17c7cajsn8f40891f4a76",
    'X-RapidAPI-Host': "free-nba.p.rapidapi.com"
}
stats_ = []
conn.close()

for i in range(start_iteration, 3100):
    for page in range(start_page, (i + 1) * 100):
        conn.request("GET", f"/stats?per_page=100&page={page}", headers=headers)
        res = conn.getresponse()
        data = res.read().decode("utf-8")
        try:
            data = json.loads(data)
            stats_378 = pd.json_normalize(data['data'])
            stats_list_378.append(stats_378)
        except json.JSONDecodeError:
            print(f"Error decoding page {page}")
            time.sleep(5)
    print(f"Finished iteration {i}")
    start_page = (i + 1) * 100
    time.sleep(1)

Finished iteration 378
Error decoding page 37915
Error decoding page 37917
Error decoding page 37918
Error decoding page 37922
Error decoding page 37923
Finished iteration 379
Error decoding page 38026
Error decoding page 38031
Error decoding page 38032
Error decoding page 38033
Error decoding page 38036
Finished iteration 380
Error decoding page 38142
Error decoding page 38143
Error decoding page 38144
Error decoding page 38148
Error decoding page 38150
Finished iteration 381
Error decoding page 38265
Error decoding page 38266
Error decoding page 38270
Error decoding page 38271
Error decoding page 38274
Finished iteration 382
Error decoding page 38392
Error decoding page 38394
Error decoding page 38396
Error decoding page 38398
Finished iteration 383
Error decoding page 38494
Error decoding page 38497
Error decoding page 38498
Error decoding page 38499
Finished iteration 384
Error decoding page 38500
Error decoding page 38501
Error decoding page 38593
Finished iteration 385
Finished i

Error decoding page 44245
Error decoding page 44247
Error decoding page 44248
Error decoding page 44249
Error decoding page 44260
Finished iteration 442
Error decoding page 44370
Error decoding page 44372
Error decoding page 44375
Error decoding page 44376
Error decoding page 44377
Finished iteration 443
Error decoding page 44454
Finished iteration 444
Finished iteration 445
Error decoding page 44626
Error decoding page 44627
Error decoding page 44629
Error decoding page 44631
Error decoding page 44632
Error decoding page 44633
Finished iteration 446
Error decoding page 44734
Error decoding page 44735
Error decoding page 44738
Finished iteration 447
Error decoding page 44848
Error decoding page 44849
Error decoding page 44850
Error decoding page 44852
Error decoding page 44854
Finished iteration 448
Error decoding page 44970
Error decoding page 44971
Error decoding page 44972
Error decoding page 44973
Finished iteration 449
Error decoding page 45088
Error decoding page 45091
Error deco

Error decoding page 50883
Error decoding page 50885
Error decoding page 50887
Error decoding page 50888
Finished iteration 508
Error decoding page 50996
Error decoding page 50998
Finished iteration 509
Error decoding page 51000
Error decoding page 51002
Error decoding page 51003
Finished iteration 510
Error decoding page 51114
Error decoding page 51115
Error decoding page 51117
Error decoding page 51119
Error decoding page 51120
Finished iteration 511
Error decoding page 51220
Error decoding page 51222
Error decoding page 51223
Error decoding page 51225
Error decoding page 51226
Finished iteration 512
Error decoding page 51332
Error decoding page 51337
Error decoding page 51338
Error decoding page 51339
Error decoding page 51341
Error decoding page 51343
Finished iteration 513
Error decoding page 51449
Error decoding page 51451
Error decoding page 51455
Error decoding page 51456
Error decoding page 51458
Finished iteration 514
Error decoding page 51561
Error decoding page 51563
Error d

Error decoding page 57160
Error decoding page 57161
Error decoding page 57162
Finished iteration 571
Error decoding page 57254
Error decoding page 57255
Error decoding page 57260
Error decoding page 57261
Error decoding page 57262
Error decoding page 57263
Finished iteration 572
Error decoding page 57374
Error decoding page 57375
Error decoding page 57376
Error decoding page 57377
Error decoding page 57378
Finished iteration 573
Finished iteration 574
Error decoding page 57541
Error decoding page 57542
Error decoding page 57546
Error decoding page 57547
Finished iteration 575
Error decoding page 57660
Error decoding page 57662
Error decoding page 57663
Error decoding page 57665
Finished iteration 576
Error decoding page 57776
Error decoding page 57778
Finished iteration 577
Error decoding page 57852
Finished iteration 578
Finished iteration 579
Error decoding page 58059
Error decoding page 58060
Error decoding page 58061
Error decoding page 58062
Error decoding page 58063
Finished iter

Finished iteration 616
Error decoding page 61746
Error decoding page 61748
Error decoding page 61749
Error decoding page 61750
Error decoding page 61751
Error decoding page 61752
Error decoding page 61753
Finished iteration 617
Finished iteration 618
Error decoding page 61961
Error decoding page 61964
Error decoding page 61965
Error decoding page 61967
Error decoding page 61971
Error decoding page 61972
Finished iteration 619
Error decoding page 62082
Error decoding page 62084
Error decoding page 62085
Error decoding page 62087
Error decoding page 62088
Finished iteration 620
Finished iteration 621
Finished iteration 622
Error decoding page 62308
Error decoding page 62309
Error decoding page 62310
Error decoding page 62312
Error decoding page 62315
Error decoding page 62316
Finished iteration 623
Error decoding page 62420
Error decoding page 62424
Error decoding page 62425
Error decoding page 62429
Error decoding page 62431
Finished iteration 624
Error decoding page 62549
Error decodin

...

Error decoding page 111986
Error decoding page 111987
Error decoding page 111989
Error decoding page 111991
Finished iteration 1119
Error decoding page 112097
Error decoding page 112099
Finished iteration 1120
Error decoding page 112102
Error decoding page 112103
Error decoding page 112105
Finished iteration 1121
Error decoding page 112207
Error decoding page 112208
Error decoding page 112209
Error decoding page 112211
Error decoding page 112213
Error decoding page 112215
Finished iteration 1122
Finished iteration 1123
Error decoding page 112426
Error decoding page 112430
Error decoding page 112431
Error decoding page 112433
Error decoding page 112434
Finished iteration 1124
Error decoding page 112541
Error decoding page 112542
Error decoding page 112545
Error decoding page 112546
Error decoding page 112549
Finished iteration 1125
Error decoding page 112669
Error decoding page 112671
Error decoding page 112672
Error decoding page 112673
Error decoding page 112674
Error decoding page 11

Finished iteration 1177
Error decoding page 117863
Finished iteration 1178
Finished iteration 1179
Error decoding page 118070
Error decoding page 118072
Error decoding page 118076
Finished iteration 1180
Error decoding page 118149
Finished iteration 1181
Error decoding page 118261
Finished iteration 1182
Error decoding page 118379
Error decoding page 118380
Error decoding page 118381
Error decoding page 118383
Error decoding page 118384
Finished iteration 1183
Error decoding page 118483
Error decoding page 118485
Error decoding page 118486
Error decoding page 118489
Error decoding page 118490
Error decoding page 118491
Finished iteration 1184
Error decoding page 118588
Error decoding page 118590
Error decoding page 118595
Error decoding page 118596
Error decoding page 118598
Error decoding page 118599
Finished iteration 1185
Finished iteration 1186
Error decoding page 118714
Error decoding page 118716
Error decoding page 118718
Error decoding page 118721
Error decoding page 118722
Erro

Error decoding page 123957
Error decoding page 123959
Error decoding page 123961
Finished iteration 1239
Error decoding page 124053
Error decoding page 124054
Error decoding page 124059
Error decoding page 124063
Error decoding page 124064
Finished iteration 1240
Error decoding page 124176
Error decoding page 124178
Error decoding page 124180
Error decoding page 124181
Error decoding page 124182
Finished iteration 1241
Error decoding page 124285
Error decoding page 124286
Error decoding page 124288
Error decoding page 124289
Error decoding page 124291
Finished iteration 1242
Finished iteration 1243
Error decoding page 124401
Error decoding page 124403
Error decoding page 124404
Error decoding page 124405
Error decoding page 124407
Finished iteration 1244
Error decoding page 124525
Error decoding page 124526
Error decoding page 124527
Error decoding page 124528
Error decoding page 124529
Error decoding page 124531
Finished iteration 1245
Error decoding page 124636
Error decoding page 12

Error decoding page 129696
Error decoding page 129697
Error decoding page 129698
Finished iteration 1296
Finished iteration 1297
Error decoding page 129812
Error decoding page 129814
Error decoding page 129816
Finished iteration 1298
Error decoding page 129937
Error decoding page 129938
Error decoding page 129939
Error decoding page 129940
Error decoding page 129941
Finished iteration 1299
Error decoding page 130053
Error decoding page 130054
Error decoding page 130055
Error decoding page 130060
Error decoding page 130061
Finished iteration 1300
Error decoding page 130170
Error decoding page 130174
Error decoding page 130175
Error decoding page 130177
Error decoding page 130180
Finished iteration 1301
Error decoding page 130287
Error decoding page 130288
Error decoding page 130289
Error decoding page 130290
Error decoding page 130291
Error decoding page 130293
Finished iteration 1302
Finished iteration 1303
Error decoding page 130411
Error decoding page 130412
Error decoding page 13041


KeyboardInterrupt



In [2]:
df = pd.concat(df_list, ignore_index=True) 
df.describe()

NameError: name 'pd' is not defined